In [47]:

import pandas as pd


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}

    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping


        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)

        return X_encoded

In [50]:
syn_df=pd.read_csv('drive/My Drive/synthetic_data_GAN_1_70per_training_dataset5000_Feb_2024_set26.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,0.416266,0.0,0.416266,0.416266,0.000000,-0.366995,0.416266,0.416266,0.000000,0.942092,0.942092,0.000000,-0.366995,0.942092,-0.366995,0.000000,0.000000,-0.366995,0.000000,0.000000
1,0.416266,0.0,0.416266,-0.366995,0.000000,-0.366995,0.416266,0.000000,-0.366995,-0.366995,-0.366995,-0.366995,-0.366995,0.000000,0.000000,0.416266,0.000000,-0.366995,0.416266,0.000000
2,0.416266,0.0,0.000000,-0.366995,0.416266,-0.366995,1.373052,0.000000,0.000000,0.000000,0.000000,0.416266,-0.366995,0.416266,0.000000,-0.366995,0.416266,1.373052,-0.366995,0.000000
3,0.416266,0.0,0.000000,0.416266,-0.366995,0.416266,-0.366995,0.942092,0.000000,0.942092,0.416266,0.000000,-0.366995,-0.366995,-0.366995,0.000000,0.000000,-0.366995,0.000000,0.416266
4,0.416266,0.0,0.000000,0.416266,0.000000,0.000000,0.942092,0.000000,-0.366995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.366995,0.416266,0.000000,-0.366995,-0.366995


In [51]:
org_test_df=pd.read_csv('drive/My Drive/Original_data_GAN_1_30per_testing_dataseat5000_Feb_2024.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,-0.358999,0.039822,0.031013,0.200916,1.810602,-0.605655,1.410377,0.973277,0.272236,0.973277,-0.358999,-0.928147,0.000000,0.441866,0.242130,0.441866,-0.276281,-0.203610,-0.308443,0.201986
1,1.410377,-0.111723,-0.018455,0.200916,-0.462942,0.221536,0.000000,0.000000,-0.643134,0.000000,0.000000,-0.864143,-0.557091,0.326591,0.242130,0.326591,-0.249514,-0.077157,-0.308443,0.201986
2,-0.358999,-0.358999,0.004923,0.000000,-0.358999,-0.647981,0.000000,-0.358999,0.409120,-0.358999,-0.358999,0.172797,0.622058,0.289240,-0.284555,0.289240,-0.249514,0.480250,-0.308443,-0.872765
3,0.000000,0.900476,-0.018455,0.200916,-0.242985,0.517807,0.323957,-0.242985,0.683962,0.000000,0.000000,0.441866,0.323957,0.000000,0.242130,0.000000,-0.249514,-0.214692,-0.308443,0.201986
4,-0.358999,0.323957,-0.018455,0.200916,-0.358999,-0.013731,0.000000,-0.242985,0.000000,-0.557091,0.000000,-0.561457,-0.358999,0.000000,-0.284555,0.000000,-0.249514,-0.688589,-0.308443,0.201986


In [52]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10, n_repeats=3, random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1269 24 94 113
pod:  0.5458937198067633
pof:  0.018561484918793503
AUC:  0.7636661174439849
recall:  0.5458937198067633
precision:  0.8248175182481752
F1-Score:  0.6569767441860466
accuracy:  0.9213333333333333


In [53]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1291 2 147 60
pod:  0.2898550724637681
pof:  0.0015467904098994587
AUC:  0.6441541410269344
recall:  0.2898550724637681
precision:  0.967741935483871
F1-Score:  0.44609665427509293
accuracy:  0.9006666666666666


In [54]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1293 0 206 1
pod:  0.004830917874396135
pof:  0.0
AUC:  0.5024154589371981
recall:  0.004830917874396135
precision:  1.0
F1-Score:  0.009615384615384614
accuracy:  0.8626666666666667


In [55]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1293 0 203 4
pod:  0.01932367149758454
pof:  0.0
AUC:  0.5096618357487923
recall:  0.01932367149758454
precision:  1.0
F1-Score:  0.03791469194312796
accuracy:  0.8646666666666667


In [56]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1135 158 135 72
pod:  0.34782608695652173
pof:  0.12219644238205724
AUC:  0.6128148222872323
recall:  0.34782608695652173
precision:  0.3130434782608696
F1-Score:  0.3295194508009154
accuracy:  0.8046666666666666


In [57]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1293 0 192 15
pod:  0.07246376811594203
pof:  0.0
AUC:  0.5362318840579711
recall:  0.07246376811594203
precision:  1.0
F1-Score:  0.13513513513513511
accuracy:  0.872


In [58]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0)

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1293 0 194 13
pod:  0.06280193236714976
pof:  0.0
AUC:  0.5314009661835749
recall:  0.06280193236714976
precision:  1.0
F1-Score:  0.11818181818181818
accuracy:  0.8706666666666667


In [59]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Epoch 1/30
88/88 [==============================] - 2s 7ms/step - loss: 0.3394 - accuracy: 0.8825 - val_loss: 0.2425 - val_accuracy: 0.9057
Epoch 2/30
88/88 [==============================] - 0s 5ms/step - loss: 0.2122 - accuracy: 0.9200 - val_loss: 0.1854 - val_accuracy: 0.9271
Epoch 3/30
88/88 [==============================] - 0s 5ms/step - loss: 0.1742 - accuracy: 0.9264 - val_loss: 0.1717 - val_accuracy: 0.9343
Epoch 4/30
88/88 [==============================] - 1s 7ms/step - loss: 0.1505 - accuracy: 0.9396 - val_loss: 0.1565 - val_accuracy: 0.9357
Epoch 5/30
88/88 [==============================] - 1s 8ms/step - loss: 0.1332 - accuracy: 0.9493 - val_loss: 0.1347 - val_accuracy: 0.9543
Epoch 6/30
88/88 [==============================] - 1s 9ms/step - loss: 0.1248 - accuracy: 0.9529 - val_loss: 0.1278 - val_accuracy: 0.9557
Epoch 7/30
88/88 [==============================] - 1s 8ms/step - loss: 0.1138 - accuracy: 0.9557 - val_loss: 0.1319 - val_accuracy: 0.9557
Epoch 8/30
88/88 [==

In [60]:
#RNN
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

import numpy as np
max_features = 10000 # number of words to consider as features
#create model
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
#model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
history = model.fit(X_encoded, y_syn_df, epochs=5, batch_size=128, validation_split=0.2)
y2_pred = model.predict(X_encoded_test)
y22_pred=y2_pred.round()


from sklearn.metrics import confusion_matrix
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 dense_23 (Dense)            (None, 1)                 33        
                                                                 
Total params: 328353 (1.25 MB)
Trainable params: 328353 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
22/22 [==============================] - 3s 56ms/step - loss: 0.6026 - accuracy: 0.8614 - val_loss: 0.4624 - val_accuracy: 0.8529
Epoch 2/5
22/22 [==============================] - 0s 8ms/step - loss: 0.3873 - accuracy: 0.8618 - val_loss: 0.3777 - val_accuracy: 0.852